In [6]:
from matplotlib import pyplot as plt
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [7]:
def find_files(folder_path):
    """
    Recursively find all files in a folder and its subfolders.
    
    Args:
    - folder_path (str): Path to the folder to search.
    
    Returns:
    - file_paths (list): List of paths to all files found.
    """
    file_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_paths.append(os.path.join(root, file))
    return file_paths

In [8]:
ses_dir ='databases/SES' #'databases/RFMiD/Test_Set/Test'
#get filepaths for all files in that directory and subdirectories
ses_paths = find_files(ses_dir)
rfmid_dir = 'databases/RFMiD/Test_Set/Test'
rfmid_paths = find_files(rfmid_dir)
#pick the first 15 from each and combine them
ses_paths = ses_paths[:15]
rfmid_paths = rfmid_paths[:15]
all_paths = ses_paths + rfmid_paths
#shuffle the list
np.random.shuffle(all_paths)

In [9]:
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch
model_type = "vit_t"
sam_checkpoint = "./mobile_sam.pt"

device = "cuda" if torch.cuda.is_available() else "cpu"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()
predictor = SamPredictor(mobile_sam)


c:\Users\elyes\miniconda3\envs\ird_deep_learning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\elyes\miniconda3\envs\ird_deep_learning\Lib\site-packages\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\Users\elyes\miniconda3\envs\ird_deep_learning\Lib\site-packages\mobile_sam\modeling\tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with mobile_sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapp

In [11]:
# Create the figure and axes
n_paths_to_plot = len(all_paths)
all_paths_shortened = all_paths[:n_paths_to_plot]
fig, axes = plt.subplots(nrows=n_paths_to_plot, ncols=5, figsize=(15, 5 * n_paths_to_plot))
fig.set_facecolor('gray')
# Iterate over each image
for i, path in enumerate(all_paths_shortened):
    # Open the image
    img = Image.open(path).convert('RGB')
    img_arr = np.array(img)
    #apply a gaussian blur to the image
    #cv2.GaussianBlur(img_arr, (3, 3), 0, img_arr)
    # Convert to numpy array
    # Set the bbox to the middle of the image
    x_top_left = img_arr.shape[1] // 4
    y_top_left = img_arr.shape[0] // 4
    x_bottom_right = 3 * img_arr.shape[1] // 4
    y_bottom_right = 3 * img_arr.shape[0] // 4
    # Get the middle of the image
    x_center = img_arr.shape[1] // 2
    y_center = img_arr.shape[0] // 2
    input_label = []
    center_points = []
    # Randomly add points to the image which are in a circle around the center with a radius of 3% of the image size
    max_radius_x = int(0.5 * (img_arr.shape[0]) / 2)
    max_radius_y = int(0.5 * (img_arr.shape[1]) / 2)
    fundus_label = 1
    for j in range(1000):
        radius_x = np.random.random() * max_radius_x
        radius_y = np.random.random() * max_radius_y
        angle = np.random.rand() * 2 * np.pi
        x = int(x_center + radius_x * np.cos(angle))
        y = int(y_center + radius_y * np.sin(angle))
        input_label.append(fundus_label)
        center_points.append([x, y])
    # Label all 4 corners of the image as background
    # Also add points which are close to the border of the image
    corner_offset = 0.08
    background__label = 0
    for j in range(500):
        x_close_r_bottom = np.random.randint(int(img_arr.shape[1] * (1 - corner_offset)), img_arr.shape[1])
        y_close_r_bottom = np.random.randint(int(img_arr.shape[0] * (1 - corner_offset)), img_arr.shape[0])
        x_close_r_top = np.random.randint(int(img_arr.shape[1] * (1 - corner_offset)), img_arr.shape[1])
        y_close_r_top = np.random.randint(0, int(img_arr.shape[0] * corner_offset))
        x_close_l_bottom = np.random.randint(0, int(img_arr.shape[1] * corner_offset))
        y_close_l_bottom = np.random.randint(int(img_arr.shape[0] * (1 - corner_offset)), img_arr.shape[0])
        x_close_l_top = np.random.randint(0, int(img_arr.shape[1] * corner_offset))
        y_close_l_top = np.random.randint(0, int(img_arr.shape[0] * corner_offset))
        input_label.extend([background__label, background__label, background__label, background__label])
        center_points.extend([[x_close_l_top, y_close_l_top], [x_close_r_top, y_close_r_top], [x_close_l_bottom, y_close_l_bottom], [x_close_r_bottom, y_close_r_bottom]])

    # Convert to numpy array
    input_label = np.array(input_label)
    center_points = np.array(center_points)
    predictor.set_image(img_arr)
    masks, quality, _ = predictor.predict(point_coords=center_points, point_labels=input_label, multimask_output=True, return_logits=False)
    best_mask = masks[np.argmax(quality)]


    # Plot the original image with points
    img_with_points = img_arr.copy()
    point_size = img_with_points.shape[0] // 100
    for label, point in zip(input_label, center_points):
        x, y = point
        if label == fundus_label:
            img_with_points[y-point_size:y+point_size, x-point_size:x+point_size] = [0, 255, 255]
        elif label == background__label:
            img_with_points[y-point_size:y+point_size, x-point_size:x+point_size] = [255, 0, 255]  # Assuming img_arr is in [0, 1] range

    #draw a circle around the center of the image
    for j in range(0, 360, 10):
        x = int(x_center + max_radius_x * np.cos(j * np.pi / 180))
        y = int(y_center + max_radius_y * np.sin(j * np.pi / 180))
        img_with_points[y-point_size:y+point_size, x-point_size:x+point_size] = [180, 65, 255]

    
    
    axes[i, 0].imshow(img_with_points)
    axes[i, 0].set_title('Image with annotation points')
    axes[i, 0].axis('off')

    # Plot the best mask
    axes[i, 1].imshow(best_mask, cmap='binary')
    axes[i, 1].set_title('Best segmentation mask')
    axes[i, 1].axis('off')

    # Plot the original image segmented with the best mask

    img_with_mask = img_arr.copy()
    img_with_mask[best_mask == 0] = [255, 255, 255]
    axes[i, 2].imshow(img_with_mask)
    axes[i, 2].set_title('Image segmented by best mask')
    axes[i, 2].axis('off')

    #copy the original image
    img_with_contour_filtered = img_arr.copy()
    best_mask_contour = best_mask.astype(np.uint8) * 255
    black_pixels = np.all(img_arr < [4, 4, 4], axis=-1)
    best_mask_contour[black_pixels] = 0
    #find contours in the mask
    contours, hirachy = cv2.findContours(best_mask_contour, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #draw the contours on the image which have a hiranachy of -1
    #filter the contours with a hirachy of -1
    contours = [contour for contour, hirachy in zip(contours, hirachy[0]) if hirachy[2] < 0]
    #only keep the largest contour
    contour = max(contours, key=cv2.contourArea)
    #only keep the pixels inside the contour
    mask = np.zeros_like(img_arr).astype(np.uint8)
    cv2.drawContours(mask, [contour], -1, (205,25,25), thickness=cv2.FILLED)
    img_with_contour_filtered = np.where(mask == 0, [255, 255, 255], img_with_contour_filtered)
    axes[i, 3].imshow(img_with_contour_filtered)
    axes[i, 3].set_title('Image segmented by \n largest contour in mask')
    axes[i, 3].axis('off')
    #get the max x and y values of the colored pixels in the contour mask
    colored_pixels = np.any(img_with_contour_filtered != [255, 255, 255], axis=-1)
    x_max = np.max(np.where(colored_pixels)[0])
    x_min = np.min(np.where(colored_pixels)[0])
    y_max = np.max(np.where(colored_pixels)[1])
    y_min = np.min(np.where(colored_pixels)[1])
    #draw the colored pixels in the smallest rectangle possible
    img_with_contour_filtered = img_with_contour_filtered[x_min:x_max, y_min:y_max]
    #turn all the white pixels to black which are not part of the contour
    axes[i, 4].imshow(img_with_contour_filtered)
    axes[i, 4].set_title('End result')
    axes[i, 4].axis('off')





# Adjust the layout and show the plot
plt.tight_layout()
plt.show()
